<a href="https://colab.research.google.com/github/oguzsenna/urban_sounds_classification_data_lists/blob/main/urban_sounds_classification_onisleme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import data from github repo

In [39]:
import pandas as pd
import librosa
import librosa.display
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 
import glob
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
pwd

'/content'

In [ ]:
! git clone https://github.com/Hijikata1835/repo/

fatal: destination path 'repo' already exists and is not an empty directory.


In [ ]:
cd repo/spectrograms

/content/repo/spectrograms


In [ ]:
ls

0/  2/  4/  6/  8/  UrbanSound8K.csv         urban_sounds_classification.pdf
1/  3/  5/  7/  9/  UrbanSound8K_README.txt


In [ ]:
metadata = pd.read_csv('/content/repo/spectrograms/UrbanSound8K.csv')
metadata.drop(columns = ['fsID','start', 'end','salience','fold'], inplace=True)
metadata.head()
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8732 entries, 0 to 8731
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   slice_file_name  8732 non-null   object
 1   classID          8732 non-null   int64 
 2   class            8732 non-null   object
dtypes: int64(1), object(2)
memory usage: 204.8+ KB


In [ ]:
print(metadata.classID.value_counts())

3    1000
2    1000
0    1000
9    1000
5    1000
7    1000
4    1000
8     929
1     429
6     374
Name: classID, dtype: int64


In [ ]:
def one_function_to_rule_them_all(n): # fonksiyona directoryleri koyacagiz grayscale, resize, normalize edip bize cikti verecek

  #imageFile = cv2.imread(n)
  gray = cv2.cvtColor(n, cv2.COLOR_BGR2GRAY) 

  # grayscale
  #image = cv2.imread('/content/repo/spectrograms/0/100852-0-0-0.png')
  #ray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  #print(gray.shape)
  #print('Gray Dimensions : ',gray.shape)
  #cv2_imshow(gray)

  #resizing 
  scale_percent = 50 # percent of original size
  width = int(gray.shape[1] * scale_percent / 100)
  height = int(gray.shape[0] * scale_percent / 100)
  dim = (width, height)
    
  resized = cv2.resize(gray, dim, interpolation = cv2.INTER_AREA)
  
  #print('Resized Dimensions : ',resized.shape)
  
  #cv2_imshow(resized)

  #normalize

  #setting the array for resulting image after normalization
  #resultimage = np.zeros((800, 800))
  #normalizing the given image using normalize() function
  #normalizedimage = cv2.normalize(resized,resultimage, 0, 100, cv2.NORM_MINMAX)

  #print('Normalized resized gray image', normalizedimage.shape)
  #cv2_imshow(normalizedimage)

  return resized

In [ ]:

cv_img = []
for i in range(10): 
  for img in glob.glob("/content/repo/spectrograms/"+str(i)+"/*.png"):
      n= cv2.imread(img)
      m=one_function_to_rule_them_all(n)
      m= m/255
      cv_img.append([m,i])

In [ ]:
df=pd.DataFrame(cv_img, columns= ['img', 'category'])
df[:1]

,img,category
0,"[[1.0, 1.0, 0.4588235294117647, 0.0, 0.0, 0.0,...",0


In [ ]:
X= df['img']

y= df['category']


In [ ]:
X_train, X_temporary, y_train, y_temporary = train_test_split(X,y,train_size=0.8)

X_val,X_test, y_val, y_test = train_test_split(X_temporary, y_temporary, train_size=0.5)

In [ ]:
print(f'Length of the dataset: {len(X)}')
print(f'Length of the training dataset:  {len(X_train)}')
print(f'Length of the validation dataset: {len(X_val)}')
print(f'Length of the test dataset: {len(X_test)}')

Length of the dataset: 8732
Length of the training dataset:  6985
Length of the validation dataset: 873
Length of the test dataset: 874


In [ ]:
with open('X_train.pickle', 'wb') as f:
    pickle.dump(X_train, f)

In [ ]:
with open('X_test.pickle', 'wb') as f:
    pickle.dump(X_test, f)

In [ ]:
with open('X_val.pickle', 'wb') as f:
    pickle.dump(X_val, f)

In [ ]:
with open('y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)

In [ ]:
with open('y_test.pickle', 'wb') as f:
    pickle.dump(y_test, f)

In [ ]:
with open('y_val.pickle', 'wb') as f:
    pickle.dump(y_val, f)